## 1. Import libraries

In [ ]:
import warnings
import pandas as pd
import numpy as np
import random
import datetime as dt
from sklearn.mixture import GaussianMixture
warnings.simplefilter(action='ignore', category=FutureWarning)

## 2. Input files (Spend, Name normalizer, Risk)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.10f' % x)
spend_input = pd.read_csv('C:/Users/mpadmara/FinalSpend.csv')  #Load the spend file in the desired format as mentioned in the SOP
Normalizer = pd.read_csv("C:/Users/mpadmara/Downloads/Name Normalization Input.csv")  # Load the Name Normalization file in the desired format as mentioned in the SOP
risk_user_input = pd.read_excel("C:/Users/mpadmara/Downloads/Risk Input.xlsx")  # Load the risk input file in the desired format as mentioned in the SOP
spend_input = spend_input[spend_input.columns[spend_input.columns.isin(
    ['amount', 'business_country', 'business_region', 'category''l1_category', 'l2_category',
     'Months in date', 'category', 'department', 'id', 'is_preferred', 'is_diverse', 'is_managed',
     'normalized_vendor_name', 'original_amount', 'original_currency', 'original_vendor_name', 'subcategory'])]] # Select the mentioned columns

## 3. Data Preprocessing

### 3.1 Transforming amount values

In [ ]:
spend_input['amount'] = spend_input['amount'].astype(str)
spend_input['original_amount'] = spend_input['original_amount'].astype(str)
spend_input['amount'] = spend_input.amount.str.split(',').str.join('')
spend_input['original_amount'] = spend_input.original_amount.str.split(',').str.join('')
spend_input['amount'] = pd.to_numeric(spend_input['amount'], errors='coerce')
spend_input['original_amount'] = pd.to_numeric(spend_input['original_amount'], errors='coerce')

### 3.2 Replace Not Provided to Null for business country

In [ ]:
spend_input['business_country'].replace('NOT PROVIDED', np.NaN,
                                        inplace=True)  # Replace Not Provided with null for future imputation

### 3.3 Impute missing Business Regions based on currency

In [ ]:
spend_input['business_region'] = \
spend_input.groupby('original_currency').transform(lambda x: x.fillna(x.value_counts().index[0]))[
    'business_region']  # impute business region based on currency

### 3.4 Impute missing Business Country based on business region

In [ ]:
spend_input['business_country'] = \
spend_input.groupby('business_region').transform(lambda x: x.fillna(x.value_counts().index[0]))[
    'business_country']  # impute country based on region

### 3.5 Impute missing Department based on category

In [ ]:
spend_input['department'] = \
spend_input.groupby('l2_category').transform(lambda x: x.fillna(x.value_counts().index[0]))[
    'department']  # impute department based on category

### 3.6 Replace cities and acronynms with actual Business Countries

In [ ]:
vals_to_replace = {'GBR': 'GREAT BRITAIN', 'USA': 'UNITED STATES', 'US': 'UNITED STATES', 'HONG KONG': 'CHINA',
                   'BERMUDA': 'GREAT BRITAIN', 'MIAMI': 'UNITED STATES', 'TAIWAN': 'CHINA',
                   'PUERTO RICO': 'UNITED STATES', 'MACAU': 'CHINA'}
spend_input['business_country'].replace(to_replace=vals_to_replace, inplace=True)  # Correct country mapping

### 3.7 Identify and remove erroneous vendor names

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
list_of_vendors_as_strings = [str(i) for i in set(list(spend_input['normalized_vendor_name']))]
vendor_list = [i for i in list_of_vendors_as_strings]
isenglish = [isEnglish(i) for i in list_of_vendors_as_strings]
vendor_df = pd.DataFrame(list(zip(vendor_list, isenglish)), columns=['normalized_vendor_name', 'flag'])

spend_input_vendorflag = spend_input.merge(vendor_df, how='left', on=['normalized_vendor_name'])
spend_input_updated_vendors = spend_input_vendorflag[spend_input_vendorflag['flag'] == True]


### 3.8 Remove claims related expenses and other errorneous vendors

In [ ]:
spend_input_remove_claimexpenses = spend_input_updated_vendors[
    spend_input_updated_vendors['subcategory'] != 'CLAIMS RELATED EXPENSES']
filter_list = ['00000', '00', '02 P', '1', '0001', '2', '3', '4', '004', '5', '6', '7', '8', '9', '10', '11', '13',
               '00014', '16', '17', '18', '21', '24', '25', '26', '30', '42', '43', '44', '45', '46', '49', '50', '58',
               '66', '67', '81', '82', '83', '95', '99', '105', '111', '146', '165', '170', '205', '220', '241',
               '00358', '379', '401', '402', '407', '428', '484', '557', '00610', '00611', '616', '618', '638', '711',
               '0000000724', '759', '825', '857', '1094', '0000001140', '0000001143', '1212', '1250', '1268', '1418',
               '1424', '1444', '1720', '1741', '1760', '1810', '1823', '1826', '1831', '1852', '1862',
               '1880', '1908', '1909', '1919', '1920', '1933', '1935', '1951', '1957', '1962', '1965', '2127', '2248',
               '3106', '3412', '3414', '3416', '3531', '3564', '4278', '4760', '5050', '6207', '0000006291',
               '0007739', '12306', '16122', '24166', '30113', '34739', '35886', '37965', '41100', '41871', '41874',
               '7/9/2019', '11/16/19', '10/6/2020', '44193', 'MAY 20', 'AUG 1', 'OCTOBER28', '51165', '51450',
               '51769', '59941', '60111', '61255', '69482', '72109', '73310', '73460', '73748', '73769', '73882',
               '73935', '74019', '74024', '74067', '74106', '74387', '74416', '74561', '75143', '75145', '75383',
               '75413', '75992', '76020', '76406', '76409', '76684', '820085', '1034260', '1066147', '1091540',
               '1095050', '1095704', '1635964', '5985346', '10685122', '460775834', '845188728', '6042410000',
               '73690600634', '00720000290562', '2000011497406', '5E73', '#120', ',', '..', '...', '......',
               '?????????', '+++++++', 'N A', 'NO VENDOR NAME PROVIDED', 'NOT', 'NOT APPLICABLE']
spend_input_filtered = spend_input_remove_claimexpenses[
    spend_input_remove_claimexpenses.normalized_vendor_name.isin(filter_list) == False]

### 3.9 Perform vendor name normalization based on normalization input 

In [ ]:
spend_input_name_normalization = spend_input_filtered.merge(Normalizer, how='left',
                                                            on=['normalized_vendor_name', 'category'])
spend_input_post_namenormalization = spend_input_name_normalization
spend_input_post_namenormalization['normalized_vendor_name'] = np.where(
    spend_input_post_namenormalization['Normalized'].isnull(),
    spend_input_post_namenormalization['normalized_vendor_name'], spend_input_post_namenormalization['Normalized'])

## 4. Modeling

### 4.1 Model Inputs 

In [ ]:
final_assignment = pd.DataFrame()
dates = []
spend_input_post_namenormalization["Months in date"] = pd.to_datetime(
    spend_input_post_namenormalization["Months in date"])
spend_input_post_namenormalization['months'] = spend_input_post_namenormalization['Months in date'].dt.to_period('m')
end_month = spend_input_post_namenormalization['months'].max()
start_month = end_month - 12
while start_month <= end_month:
    dates.append(start_month.to_timestamp(freq='M'))
    start_month += 3

### 4.1.1 Input spend weights (0-100)

In [ ]:
spend_weight = []
n = int(input("Enter the number of weights  "))
print("\n")
for i in range(0, n):
    print("Enter the weight at index", i, "from 0-100" )
    item = int(input())
    spend_weight.append(item)
print(spend_weight)

### 4.1.2 Input Pareto contribution percent (0-100)

In [ ]:
percent_input = []
n = int(input("Enter the number of iterations for pareto"))
print("\n")
for i in range(0, n):
    print("Enter number at index", i,"from 0-100" )
    item = int(input())
    percent_input.append(item)
print(percent_input)

### 4.1.3 Input recency of data in days

In [ ]:
days_input = []
n = int(input("Enter the number of months for recency"))
print("\n")
for i in range(0, n):
    print("Enter number at index", i,"(in days)" )
    item = int(input())
    days_input.append(item)
print(days_input)

### 4.2 Clustering of vendors based on spend and risk for all categories

In [ ]:
for date in dates:
    spend_input_post_namenormlaization_quarter = spend_input_post_namenormalization[
        spend_input_post_namenormalization['Months in date'] <= date]
    spend_input_datesgroup = spend_input_post_namenormlaization_quarter.groupby(['category', 'normalized_vendor_name'])
    spend_input_recency = spend_input_datesgroup.agg(minimum_Date=('Months in date', np.min),
                                                     maximum_date=('Months in date', np.max))
    spend_input_recency['latest_date'] = spend_input_recency['maximum_date'].max()
    spend_input_recency['difference_From_latest'] = spend_input_recency['latest_date'] - spend_input_recency[
        'maximum_date']
    spend_input_recency['difference_From_latest'] = spend_input_recency['difference_From_latest'].dt.days
    
    for days in days_input:
        vendors_latest = spend_input_recency[spend_input_recency['difference_From_latest'] <= days]
        vendors_latest.reset_index(inplace=True)
        vendors_latest_flagged = spend_input_post_namenormlaization_quarter.merge(
            vendors_latest[['category', 'normalized_vendor_name', 'difference_From_latest']], how='left',
            on=['category', 'normalized_vendor_name'])
        vendors_latest_filtered = vendors_latest_flagged[vendors_latest_flagged['difference_From_latest'].notnull()]
        vendors_latest_filtered_grouped = vendors_latest_filtered.groupby(['category', 'normalized_vendor_name']).sum()
        vendors_latest_filtered_grouped.reset_index(inplace=True)
        vendors_latest_filtered_grouped = vendors_latest_filtered_grouped.rename(
            columns={'amount': 'grouped_amount', 'original_amount': 'grouped_original_amount'})
        vendors_latest_filtered_totals = vendors_latest_filtered.merge(vendors_latest_filtered_grouped[
                                                                           ['category', 'normalized_vendor_name',
                                                                            'grouped_amount',
                                                                            'grouped_original_amount']], how='left',
                                                                       on=['category', 'normalized_vendor_name'])
        vendors_latest_filtered_totals['contribution'] = vendors_latest_filtered_totals['grouped_amount'] * 100 / \
                                                         vendors_latest_filtered_totals.groupby(['category'])[
                                                             'grouped_amount'].transform('sum')
        vendors_latest_filtered_totals_sorted = vendors_latest_filtered_totals.groupby('category').apply(
            lambda x: x.sort_values(by='contribution', ascending=False).reset_index(drop=True))
        subcat_analysis = vendors_latest_filtered_totals_sorted.drop('category', axis=1).reset_index()
        vendors_latest_filtered_totals_sorted_unique = subcat_analysis[
            ['category', 'normalized_vendor_name', 'grouped_amount']].drop_duplicates()
        vendors_latest_filtered_totals_sorted_unique['contribution'] = vendors_latest_filtered_totals_sorted_unique[
                                                                           'grouped_amount'] * 100 / \
                                                                       vendors_latest_filtered_totals_sorted_unique.groupby(
                                                                           ['category'])['grouped_amount'].transform(
                                                                           'sum')
        vendors_latest_filtered_totals_sorted_unique['cumsum'] = \
        vendors_latest_filtered_totals_sorted_unique.groupby('category')['contribution'].transform('cumsum')
        
        for percent in percent_input:
            pareto = vendors_latest_filtered_totals_sorted_unique[
                vendors_latest_filtered_totals_sorted_unique['cumsum'] <= percent]
            pareto_subcat = pareto.merge(subcat_analysis[['category', 'normalized_vendor_name', 'subcategory']],
                                         on=['category', 'normalized_vendor_name'], how='left').drop_duplicates()
            subcat = set(list(pareto_subcat['subcategory']))
            subcat_exhaustive = set(list(subcat_analysis['subcategory']))
            missing_subcat = list(subcat_exhaustive - subcat)
            missingsubcat_df = subcat_analysis[subcat_analysis['subcategory'].isin(missing_subcat)]
            missingsubcat_grouped = missingsubcat_df.groupby(
                ['category', 'normalized_vendor_name', 'subcategory']).sum()
            missingsubcat_grouped.reset_index(inplace=True)
            appendmissing_subcat = missingsubcat_grouped[
                missingsubcat_grouped['amount'] == missingsubcat_grouped.groupby('subcategory')['amount'].transform(
                    'max')][['category', 'normalized_vendor_name', 'amount']]
            appendmissing_subcat = appendmissing_subcat.rename(columns={'amount': 'grouped_amount'})
            spend = pd.concat([pareto, appendmissing_subcat], axis=0)
            risk_input = pd.DataFrame(list(spend['normalized_vendor_name'].drop_duplicates()),
                                      columns=['normalized_vendor_name'])
            risk = risk_input.merge(risk_user_input, on='normalized_vendor_name', how='left')
            random.seed(10)
            risk['Cyber Risk Score input'] = [random.randint(1, 6) for k in risk.index]
            risk['Resiliency Risk Score input'] = [random.randint(1, 6) for k in risk.index]
            risk['Compliance Risk Score input'] = [random.randint(1, 6) for k in risk.index]
            risk['Data Privacy Risk Score input'] = [random.randint(1, 6) for k in risk.index]
            risk['Financial Risk Score input'] = [random.randint(1, 6) for k in risk.index]
            risk['ESG input'] = [random.randint(1, 6) for k in risk.index]
            risk['Cyber Risk Score'].fillna(risk['Cyber Risk Score input'], inplace=True)
            risk['Resiliency Risk Score'].fillna(risk['Resiliency Risk Score input'], inplace=True)
            risk['Compliance Risk Score'].fillna(risk['Compliance Risk Score input'], inplace=True)
            risk['Data Privacy Risk Score'].fillna(risk['Data Privacy Risk Score input'], inplace=True)
            risk['Financial Risk Score'].fillna(risk['Financial Risk Score input'], inplace=True)
            risk['ESG'].fillna(risk['ESG input'], inplace=True)
            clustering_data = spend.merge(risk, on='normalized_vendor_name', how='left')
            col_list = ['Cyber Risk Score', 'Resiliency Risk Score', 'Compliance Risk Score', 'Data Privacy Risk Score',
                        'Financial Risk Score', 'ESG']
            clustering_data['Totalrisk'] = clustering_data[col_list].sum(axis=1)
            clustering_data['Norisk'] = 10 * len(col_list) - clustering_data['Totalrisk']
            clustering_data['norisk_contribution_overall'] = clustering_data['Norisk'] / clustering_data['Norisk'].sum()
            clustering_data['spend_contribution_overall'] = clustering_data['grouped_amount'] / clustering_data[
                'grouped_amount'].sum()
            clustering_data['risk_contribution_overall'] = clustering_data['Totalrisk'] / clustering_data[
                'Totalrisk'].sum()

            for i in spend_weight:
                clustering_data['objective_spend_overall'] = (clustering_data['spend_contribution_overall']) * i
                if i == 100:
                    clustering_data['objective_norisk_overall'] = 0
                else:
                    clustering_data['objective_norisk_overall'] = clustering_data['norisk_contribution_overall'] * (
                            100 - i)
                IQR = clustering_data['objective_spend_overall'].quantile(0.75) - clustering_data[
                    'objective_spend_overall'].quantile(0.25)
                median = clustering_data['objective_spend_overall'].median()
                # clustering_data_without_outliers=clustering_data.loc[clustering_data['objective_spend_overall'] < clustering_data['objective_spend_overall'].quantile(0.99)]
                df_cons = clustering_data.sort_values(by='objective_spend_overall', ascending=False)
                clustering_data_without_outliers = df_cons.iloc[50:]
                X = clustering_data_without_outliers.loc[:,
                    ['objective_spend_overall', 'objective_norisk_overall']].values
                gmm = GaussianMixture(n_components=4,random_state=42)
                gmm.fit(X)
                labels = gmm.predict(X)
                clustering_data_without_outliers['Gaussian'] = pd.Series(labels,
                                                                         index=clustering_data_without_outliers.index)
                list_labels = list(labels)
                count = [list_labels.count(i) for i in set(list_labels)]
                count_df = pd.DataFrame(count, columns=['Count'])
                # append=clustering_data.loc[clustering_data['objective_spend_overall'] >= clustering_data['objective_spend_overall'].quantile(0.99)]
                append = clustering_data.nlargest(50, ['objective_spend_overall'])
                append['Gaussian'] = count_df['Count'].idxmin()
                cluster_assignment = pd.concat([clustering_data_without_outliers, append], axis=0)
                cluster_assignment['SpendWeight'] = i
                cluster_assignment['Pareto'] = percent
                cluster_assignment['recency'] = days
                cluster_assignment['Quarter'] = date
                final_assignment = final_assignment.append(cluster_assignment, ignore_index=True)

## 5. Post-Processing

### 5.1 Generate Tiers based on modeling results

In [ ]:
# Generate ranks that indicate tiers based on count for each gaussian label
final_assignment['counts'] = final_assignment.groupby(['Quarter','Gaussian','Pareto','recency','SpendWeight'])['SpendWeight'].transform('count')
final_assignment['Rank'] = final_assignment.groupby(['Quarter','Pareto','recency','SpendWeight'])['counts'].rank(method='dense').astype(int)

### 5.2 Output the result to populate dashboard

In [ ]:
dashboard=final_assignment.merge(spend_input_post_namenormalization[['business_country','business_region','category','normalized_vendor_name','subcategory','is_preferred','is_diverse','is_managed']].drop_duplicates(),on=['category','normalized_vendor_name'],how='left').drop_duplicates()
dashboard.to_csv('DashboardAnalysis.csv')  # Check working directory for the ouput file
spend_input_post_namenormalization.to_csv('ClusterInput.csv')